# **Evaluation of Feature Enhancement**  
  
  

## **Contents**
1. [Importing Libraries](evaluation_of_feature_enhancement.ipynb#1-importing-libraries)  
   
2. [Load Results](evaluation_of_feature_enhancement.ipynb#2-load-results)  
   
3. [RMSE and MAE](evaluation_of_feature_enhancement.ipynb#3-rmse-and-mae)  
   - 3.1 [RMSE and MAE Statistical Analysis](evaluation_of_feature_enhancement.ipynb#3.1-rmse-and-mae-statistical-analysis)  
  
4. [CG-EGA Summary Classification](evaluation_of_feature_enhancement.ipynb#4-cg-ega-summary-classification)
   - 4.1 [CG-EGA Summary Classification Statistical Analysis](evaluation_of_feature_enhancement.ipynb#4.1-cg-ega-summary-classification-statistical-analysis)  
  


## **1. Importing Libraries**

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import seaborn as sns
import torch
from IPython.display import display


current_dir = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(current_dir, "../../"))

sys.path.append(os.path.join(PROJECT_ROOT, "shared_utilities"))
try:
	from metrics import *
except ModuleNotFoundError:
	print("Module 'metrics' not found. Please ensure it exists in the 'shared_utilities' directory.")

In [2]:
ptid_list = [540, 544, 552, 559, 563, 567, 570, 575, 584, 588, 591, 596]


## **2. Load Results**

In [ ]:
"""Base Model Evaluation Files"""

base_eval_dir = os.path.join(PROJECT_ROOT, "evaluation

base_overall_dict = {}

aggregate_df = pd.DataFrame()

for ptid in ptid_list:
    base_eval_df = pd.read_csv(os.path.join(base_eval_dir, f"patient_{ptid}", "base_model_eval", f"patient_{ptid}_base_model_detailed_test.csv"))
    base_overall_dict[ptid] = base_eval_df


all_base_dfs = list(base_overall_dict.values())

aggregate_base_df = pd.concat(all_base_dfs, axis=0)
aggregate_base_df = aggregate_base_df.reset_index(drop=True)

aggregate_base_df.head()


FileNotFoundError: [Errno 2] No such file or directory: '/Users/josephpassant/BG_Prediction/models/jpformer/fine_tuning_development_files/base_model_evaluation/patient_540/base_model_eval/patient_540_base_model_detailed_test.csv'

In [ ]:
fine_tuned_dir = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/loss_function_weights_lowest")

fine_tuned_dict = {}

for ptid in ptid_list:
    fine_tuned_df = pd.read_csv(os.path.join(fine_tuned_dir, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    fine_tuned_dict[ptid] = fine_tuned_df



all_fine_tuned_dfs = list(fine_tuned_dict.values())

aggregate_fine_tuned_df = pd.concat(all_fine_tuned_dfs, axis=0)
aggregate_fine_tuned_df = aggregate_fine_tuned_df.reset_index(drop=True)

aggregate_fine_tuned_df.head()


## **Aggregate Results**

## **3. RMSE and MAPE**

In [ ]:
# Define a simple RMSE function
def calculate_rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

base_rmse = calculate_rmse(aggregate_base_df['true_glucose'], aggregate_base_df['predicted_glucose'])
fine_tuned_rmse = calculate_rmse(aggregate_fine_tuned_df['true_glucose'], aggregate_fine_tuned_df['predicted_glucose'])



def mape(predictions, targets):
    return np.mean(np.abs((predictions - targets) / targets)) * 100

base_mape = mape(aggregate_base_df['predicted_glucose'], aggregate_base_df['true_glucose'])
fine_tuned_mape = mape(aggregate_fine_tuned_df['predicted_glucose'], aggregate_fine_tuned_df['true_glucose'])


In [ ]:
# create table with model in 1st column and rmse and mae in 2nd and 3rd columns
rmse_mape_table = pd.DataFrame({'model': ['base model', 'fine-tuned models'], 'MSE': [base_rmse, fine_tuned_rmse], 'MAPE': [base_mape, fine_tuned_mape]})
rmse_mape_table


In [ ]:
# plot comparison column chart for RMSE and MAE and display in a 2x1 grid
fig, axs = plt.subplots(1, 2, figsize=(10,4))

# Define custom colors for the models
model1_color = (173 / 255, 29 / 255, 30 / 255)  # Without Feature Enhancement
model2_color = (110 / 255, 180 / 255, 186 / 255)  # With Feature Enhancement
palette = [model1_color, model2_color]

# Plot RMSE
sns.barplot(x='model', y='MSE', data=rmse_mape_table, ax=axs[0], hue='model', palette=palette, legend=False)
axs[0].set_title('RMSE', fontsize=16, fontweight='bold', pad=20)
axs[0].set_ylabel('RMSE (mg/dL)', fontsize=14)
axs[0].set_xlabel('')  # Remove x-axis label

# Plot MAE
sns.barplot(x='model', y='MAPE', data=rmse_mape_table, ax=axs[1], hue='model', palette=palette, legend=False)
axs[1].set_title('MAPE', fontsize=16, fontweight='bold', pad=20)
axs[1].set_ylabel('MAPE', fontsize=14)
axs[1].set_xlabel('')  # Remove x-axis label

# Set y-axis limits and add data labels
for ax in axs:
    ax.set_ylim(0, 50)  # Adjusted to better fit the data range
    # Make tick labels smaller
    ax.tick_params(axis='both', labelsize=14)
    # Add data labels
    for p in ax.patches:
        ax.annotate(f"{p.get_height():.1f}", 
                  (p.get_x() + p.get_width() / 2., p.get_height()),
                  ha='center', va='bottom', fontsize=14)
    # Remove top and right borders
    sns.despine(ax=ax)

plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjust to make room for the title
plt.show()


### **3.1 RMSE and MAPE Statistical Analysis**



In [ ]:
def return_ttests(no_fe_model, fe_model, glycemic_region, model1_name="Model w/o FE", model2_name="Model w/ FE"):
    print(f"\n🔍 Running t-test for {model1_name} vs {model2_name} ({glycemic_region.capitalize()}glycaemic Range Performance):\n")

    # Copy data to avoid modifying the originals
    df1 = no_fe_model.copy()
    df2 = fe_model.copy()

    # Filter based on glycemic region
    glycemic_region = glycemic_region.lower()
    if glycemic_region == 'hypo':
        df1 = df1[df1['glycemic_region'] == 'hypo']
        df2 = df2[df2['glycemic_region'] == 'hypo']
    elif glycemic_region == 'hyper':
        df1 = df1[df1['glycemic_region'] == 'hyper']
        df2 = df2[df2['glycemic_region'] == 'hyper']
    elif glycemic_region == 'eu':
        df1 = df1[df1['glycemic_region'] == 'eu']
        df2 = df2[df2['glycemic_region'] == 'eu']
    else:
        pass  # Use all data if 'overall' or invalid

    # Calculate errors
    df1['absolute_error'] = np.abs(df1['true_glucose'] - df1['predicted_glucose'])
    df2['absolute_error'] = np.abs(df2['true_glucose'] - df2['predicted_glucose'])

    df1['squared_error'] = df1['absolute_error'] ** 2
    df2['squared_error'] = df2['absolute_error'] ** 2
    
    # Calculate percentage errors for MAPE
    df1['percentage_error'] = np.abs((df1['true_glucose'] - df1['predicted_glucose']) / df1['true_glucose']) * 100
    df2['percentage_error'] = np.abs((df2['true_glucose'] - df2['predicted_glucose']) / df2['true_glucose']) * 100

    # Use Welch's t-test (unpaired, unequal variance)
    tt_mse, p_mse = stats.ttest_ind(df1['squared_error'], df2['squared_error'], equal_var=False)
    tt_mape, p_mape = stats.ttest_ind(df1['percentage_error'], df2['percentage_error'], equal_var=False)

    # Format results
    results = pd.DataFrame({
        "Metric": ["RMSE", "MAPE"],
        "t-statistic": [tt_mse, tt_mape],
        "p-value": [p_mse, p_mape],
        "Significance (p < 0.05)": [p_mse < 0.05, p_mape < 0.05]
    })

    display(results)
    return results

In [ ]:
overall_ttests = return_ttests(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region="overall", model1_name="Base Model", model2_name="Fine-tuned Model")


## **4. CG-EGA Summary Classifications**

In [ ]:
# ADD BINARY COLUM FOR ACCURATE PREDICTION BASED ON CGEGA_CLASS COLUMN
aggregate_base_df['AP'] = np.where(aggregate_base_df['CG_EGA_Class'] == 'AP', 1, 0)
aggregate_base_df['BE'] = np.where(aggregate_base_df['CG_EGA_Class'] == 'BE', 1, 0)
aggregate_base_df['EP'] = np.where(aggregate_base_df['CG_EGA_Class'] == 'EP', 1, 0)


aggregate_fine_tuned_df['AP'] = np.where(aggregate_fine_tuned_df['CG_EGA_Class'] == 'AP', 1, 0)
aggregate_fine_tuned_df['BE'] = np.where(aggregate_fine_tuned_df['CG_EGA_Class'] == 'BE', 1, 0)
aggregate_fine_tuned_df['EP'] = np.where(aggregate_fine_tuned_df['CG_EGA_Class'] == 'EP', 1, 0)

In [ ]:
# Import necessary libraries
# Create summary tables for overall, hypo, eu, and hyper glycaemic regions
regions = ['overall', 'hypo', 'eu', 'hyper']
summary_tables = {}

for region in regions:
    # Initialize empty DataFrame with specific dtypes to avoid warning
    summary_df = pd.DataFrame({
        'Model': pd.Series(dtype='object'),
        'AP': pd.Series(dtype='int64'), 
        'BE': pd.Series(dtype='int64'), 
        'EP': pd.Series(dtype='int64'), 
        'Count': pd.Series(dtype='int64'),
        'AP_pct': pd.Series(dtype='float64'), 
        'BE_pct': pd.Series(dtype='float64'), 
        'EP_pct': pd.Series(dtype='float64')
    })
    
    # Process each model
    for model, df in zip(['base model', 'fine-tuned models'], 
                        [aggregate_base_df, aggregate_fine_tuned_df]):
        
        # Filter for region if not overall
        if region != 'overall':
            region_df = df[df['glycemic_region'] == region]
        else:
            region_df = df
            
        # Calculate counts
        ap_count = region_df['AP'].sum()
        be_count = region_df['BE'].sum()
        ep_count = region_df['EP'].sum()
        total_count = len(region_df)
        
        # Create a new row as a dictionary and append it to the DataFrame
        new_row = {
            'Model': model, 
            'AP': ap_count, 
            'BE': be_count, 
            'EP': ep_count, 
            'Count': total_count,
            'AP:EP': ap_count / ep_count if ep_count != 0 else np.nan,
            'AP_pct': ap_count / total_count * 100, 
            'BE_pct': be_count / total_count * 100, 
            'EP_pct': ep_count / total_count * 100
        }
        summary_df = pd.concat([summary_df, pd.DataFrame([new_row])], ignore_index=True)
    
    # Store in dictionary
    summary_tables[region] = summary_df

# Create 1x2 grid figure with custom width ratios
fig, axs = plt.subplots(1, 2, figsize=(10, 4), gridspec_kw={'width_ratios': [1, 2]})

# PLOT 1: Overall AP:EP ratio (left panel)
sns.barplot(
    x='Model', 
    y='AP:EP', 
    data=summary_tables['overall'], 
    ax=axs[0], 
    palette=palette,
    hue='Model',
    legend=False
)
axs[0].set_title('Overall AP:EP Ratio', fontsize=16, fontweight='bold', pad=20)
axs[0].set_ylabel('AP:EP Ratio', fontsize=14)
axs[0].set_xlabel('')  # Remove x-axis label
axs[0].tick_params(axis='both', labelsize=14)
axs[0].set_ylim(0, 100)  # Set y-axis range from 0 to 100

# Add data labels
for p in axs[0].patches:
    axs[0].annotate(f"{p.get_height():.1f}", 
                  (p.get_x() + p.get_width() / 2., p.get_height()),
                  ha='center', va='bottom', fontsize=11, fontweight='bold')

# PLOT 2: AP:EP by Glycemic Region (right panel)
# Create a new dataframe that combines the regional data for plotting
regional_data = pd.concat([
    summary_tables['hypo'][['Model', 'AP:EP']].assign(Region='Hypo'),
    summary_tables['eu'][['Model', 'AP:EP']].assign(Region='Eu'),
    summary_tables['hyper'][['Model', 'AP:EP']].assign(Region='Hyper')
])

sns.barplot(
    x='Region', 
    y='AP:EP', 
    hue='Model',
    data=regional_data, 
    ax=axs[1], 
    palette=palette
)
axs[1].set_title('AP:EP Ratio Across Glycemic Regions', fontsize=14, fontweight='bold')
axs[1].set_ylabel('AP:EP Ratio', fontsize=12)
axs[1].set_xlabel('Glycemic Region', fontsize=12)
axs[1].tick_params(axis='both', labelsize=10)
axs[1].set_ylim(0, 100)  # Set y-axis range from 0 to 100

# Add data labels to the second chart
for container in axs[1].containers:
    axs[1].bar_label(container, fmt='%.1f', fontsize=11, fontweight='bold')


# Add legend to the right plot
axs[1].legend(title='Model', fontsize=9, title_fontsize=10)

# Remove top and right borders
sns.despine(ax=axs[0])
sns.despine(ax=axs[1])


plt.tight_layout()
plt.subplots_adjust(top=0.9)  # Adjust to make room for the title
plt.show()


In [ ]:
# Define custom colors for the models
model1_color = (173 / 255, 29 / 255, 30 / 255)  # Without Feature Enhancement
model2_color = (110 / 255, 180 / 255, 186 / 255)  # With Feature Enhancement
palette = [model1_color, model2_color]

# Create summary tables for overall, hypo, eu, and hyper glycaemic regions
regions = ['overall', 'hypo', 'eu', 'hyper']
summary_tables = {}

for region in regions:
    # Initialize empty DataFrame with specific dtypes to avoid warning
    summary_df = pd.DataFrame({
        'Model': pd.Series(dtype='object'),
        'AP': pd.Series(dtype='int64'), 
        'BE': pd.Series(dtype='int64'), 
        'EP': pd.Series(dtype='int64'), 
        'Count': pd.Series(dtype='int64'),
        'AP_pct': pd.Series(dtype='float64'), 
        'BE_pct': pd.Series(dtype='float64'), 
        'EP_pct': pd.Series(dtype='float64')
    })
    
    # Process each model
    for model, df in zip(['Population\nJPFormer', 'Personalised\nJPFormer'], 
                        [aggregate_base_df, aggregate_fine_tuned_df]):
        
        # Filter for region if not overall
        if region != 'overall':
            region_df = df[df['glycemic_region'] == region]
        else:
            region_df = df
            
        # Calculate counts
        ap_count = region_df['AP'].sum()
        be_count = region_df['BE'].sum()
        ep_count = region_df['EP'].sum()
        total_count = len(region_df)
        
        # Create a new row as a dictionary and append it to the DataFrame
        new_row = {
            'Model': model, 
            'AP': ap_count, 
            'BE': be_count, 
            'EP': ep_count, 
            'Count': total_count,
            'AP:EP': ap_count / ep_count if ep_count != 0 else np.nan,
            'AP_pct': ap_count / total_count * 100, 
            'BE_pct': be_count / total_count * 100, 
            'EP_pct': ep_count / total_count * 100
        }
        summary_df = pd.concat([summary_df, pd.DataFrame([new_row])], ignore_index=True)
    
    # Store in dictionary
    summary_tables[region] = summary_df

# Create 1x2 grid figure with custom width ratios
fig, axs = plt.subplots(1, 2, figsize=(10, 5), gridspec_kw={'width_ratios': [1, 2]})

# PLOT 1: Overall EP% (left panel)
sns.barplot(
    x='Model', 
    y='EP_pct', 
    data=summary_tables['overall'], 
    ax=axs[0], 
    palette=palette,
    hue='Model',
    legend=False
)
axs[0].set_title('Overall EP%', fontsize=16, fontweight='bold', pad=20)
axs[0].set_ylabel('EP%', fontsize=14)
axs[0].set_xlabel('')  # Remove x-axis label
axs[0].tick_params(axis='both', labelsize=14)
axs[0].set_ylim(0, 50)  # Set y-axis range from 0 to 50
axs[0].grid(False)  # Remove gridlines

# Add data labels
for p in axs[0].patches:
    axs[0].annotate(f"{p.get_height():.1f}%", 
                  (p.get_x() + p.get_width() / 2., p.get_height()),
                  ha='center', va='bottom', fontsize=14)

# PLOT 2: EP% by Glycemic Region (right panel)
# Create a new dataframe that combines the regional data for plotting
regional_data = pd.concat([
    summary_tables['hypo'][['Model', 'EP_pct']].assign(Region='Hypo'),
    summary_tables['eu'][['Model', 'EP_pct']].assign(Region='Eu'),
    summary_tables['hyper'][['Model', 'EP_pct']].assign(Region='Hyper')
])

sns.barplot(
    x='Region', 
    y='EP_pct', 
    hue='Model',
    data=regional_data, 
    ax=axs[1], 
    palette=palette
)
axs[1].set_title('EP% Across Glycemic Regions', fontsize=16, fontweight='bold', pad=20)
axs[1].set_ylabel('EP%', fontsize=14)
axs[1].set_xlabel('Glycemic Region', fontsize=14)
axs[1].tick_params(axis='both', labelsize=14)
axs[1].set_ylim(0, 50)  # Set y-axis range from 0 to 50
axs[1].grid(False)  # Remove gridlines

# Add data labels to the second chart
for container in axs[1].containers:
    axs[1].bar_label(container, fmt='%.1f%%', fontsize=14)

# Add legend to the right plot
axs[1].legend(title='Model', fontsize=14, title_fontsize=14)

# Remove top and right borders
sns.despine(ax=axs[0])
sns.despine(ax=axs[1])

plt.tight_layout()
plt.subplots_adjust(top=0.9)  # Adjust to make room for the title
plt.show()


In [ ]:
# Define custom colors for the models
model1_color = (173 / 255, 29 / 255, 30 / 255)  # Without Feature Enhancement
model2_color = (110 / 255, 180 / 255, 186 / 255)  # With Feature Enhancement
palette = [model1_color, model2_color]

# Create summary tables for overall, hypo, eu, and hyper glycaemic regions
regions = ['overall', 'hypo', 'eu', 'hyper']
summary_tables = {}

for region in regions:
    # Initialize empty DataFrame with specific dtypes to avoid warning
    summary_df = pd.DataFrame({
        'Model': pd.Series(dtype='object'),
        'AP': pd.Series(dtype='int64'), 
        'BE': pd.Series(dtype='int64'), 
        'EP': pd.Series(dtype='int64'), 
        'Count': pd.Series(dtype='int64'),
        'AP_pct': pd.Series(dtype='float64'), 
        'BE_pct': pd.Series(dtype='float64'), 
        'EP_pct': pd.Series(dtype='float64')
    })

    
    # Process each model
    for model, df in zip(['base', 'fine-tuned'], 
                        [aggregate_base_df, aggregate_fine_tuned_df]):
        
        
        # Filter for region if not overall
        if region != 'overall':
            region_df = df[df['glycemic_region'] == region]
        else:
            region_df = df
            
        # Calculate counts
        ap_count = region_df['AP'].sum()
        be_count = region_df['BE'].sum()
        ep_count = region_df['EP'].sum()
        total_count = len(region_df)
        
        # Create a new row as a dictionary and append it to the DataFrame
        new_row = {
            'Model': model, 
            'AP': ap_count, 
            'BE': be_count, 
            'EP': ep_count, 
            'Count': total_count,
            'AP:EP': ap_count / ep_count if ep_count != 0 else np.nan,
            'AP_pct': ap_count / total_count * 100, 
            'BE_pct': be_count / total_count * 100, 
            'EP_pct': ep_count / total_count * 100
        }
        summary_df = pd.concat([summary_df, pd.DataFrame([new_row])], ignore_index=True)
    
    # Store in dictionary
    summary_tables[region] = summary_df

# Create 1x2 grid figure with custom width ratios
fig, axs = plt.subplots(1, 2, figsize=(10, 5), gridspec_kw={'width_ratios': [1, 2]})

# PLOT 1: Overall AP% (left panel)
sns.barplot(
    x='Model', 
    y='AP_pct', 
    data=summary_tables['overall'], 
    ax=axs[0], 
    palette=palette,
    hue='Model',
    legend=False
)
axs[0].set_title('Overall AP%', fontsize=16, fontweight='bold', pad=20)
axs[0].set_ylabel('AP%', fontsize=14)
axs[0].set_xlabel('')  # Remove x-axis label
axs[0].tick_params(axis='both', labelsize=14)
axs[0].set_ylim(50, 100)  # Set y-axis range from 0 to 100

# Add data labels
for p in axs[0].patches:
    axs[0].annotate(f"{p.get_height():.1f}%", 
                  (p.get_x() + p.get_width() / 2., p.get_height()),
                  ha='center', va='bottom', fontsize=14)

# PLOT 2: AP% by Glycemic Region (right panel)
# Create a new dataframe that combines the regional data for plotting
regional_data = pd.concat([
    summary_tables['hypo'][['Model', 'AP_pct']].assign(Region='Hypo'),
    summary_tables['eu'][['Model', 'AP_pct']].assign(Region='Eu'),
    summary_tables['hyper'][['Model', 'AP_pct']].assign(Region='Hyper')
])

sns.barplot(
    x='Region', 
    y='AP_pct', 
    hue='Model',
    data=regional_data, 
    ax=axs[1], 
    palette=palette,
    legend=False
)
axs[1].set_title('AP% Across Glycemic Regions', fontsize=16, fontweight='bold', pad=20)
axs[1].set_ylabel('AP%', fontsize=14)
axs[1].set_xlabel('Glycemic Region', fontsize=14)
axs[1].tick_params(axis='both', labelsize=14)
axs[1].set_ylim(50, 100)  # Set y-axis range from 0 to 100

# Add data labels to the second chart
for container in axs[1].containers:
    axs[1].bar_label(container, fmt='%.1f%%', fontsize=14)

# Add legend to the right plot


# Remove top and right borders
sns.despine(ax=axs[0])
sns.despine(ax=axs[1])

plt.tight_layout()
plt.subplots_adjust(top=0.9)  # Adjust to make room for the title
plt.show()


In [ ]:
def create_timepoint_ega_dataframe(df, region=None):
    # Initialize a dataframe to store results with proper data types
    result_df = pd.DataFrame(index=(np.arange(24)+1) * 5)  # timepoints (n+1) * 5
    result_df.index.name = 'timepoint'
    
    # Initialize columns for percentages, counts and total
    result_df['AP_percent'] = 0.0
    result_df['BP_percent'] = 0.0
    result_df['EP_percent'] = 0.0
    result_df['AP_count'] = 0
    result_df['BP_count'] = 0
    result_df['EP_count'] = 0
    result_df['Total_count'] = 0

    
    # Create a copy of the dataframe to avoid modifying the original
    df = df.copy()

    if region == 'hypo':
        # Filter for hypoglycemic readings only
        df = df[df['glycemic_region'] == 'hypo']
    elif region == 'hyper':
        # Filter for hyperglycemic readings only
        df = df[df['glycemic_region'] == 'hyper']
    elif region == 'eu':
        # Filter for euglycemic readings only
        df = df[df['glycemic_region'] == 'eu']
    else:
        df = df  # Use all data

    # Process each timepoint
    for timepoint in range(1,24):
        # Filter data for this timepoint
        timepoint_data = df[df['timepoint'] == timepoint]
        
        if len(timepoint_data) > 0:
            # Count occurrences of each CG_EGA_Class
            class_counts = timepoint_data['CG_EGA_Class'].value_counts()
            total_points = len(timepoint_data)
            
            # Store total count for this timepoint
            result_df.loc[(timepoint + 1) * 5, 'Total_count'] = total_points
            
            # Calculate percentages and counts for AP, BP, and EP
            for class_prefix in ['A', 'B', 'E']:
                # Find all classes that start with this letter (AP, BP, EP)
                class_matches = [c for c in class_counts.index if c.startswith(class_prefix)]
                count = sum([class_counts[c] for c in class_matches if c in class_counts])
                
                # Store count
                result_df.loc[(timepoint + 1) * 5, f'{class_prefix}P_count'] = count
                
                # Store percentage
                result_df.loc[(timepoint + 1) * 5, f'{class_prefix}P_percent'] = float((count / total_points) * 100)

    result_df['AP:EP'] = result_df['AP_count'] / result_df['EP_count']


    # result_df = result_df.drop(index=0)
    # result_df = result_df.reset_index()
    # result_df['timepoint'] = result_df['timepoint'].astype(int)

    
    return result_df






In [ ]:
def return_ep_ph_chart(model1, model2, glycemic_region):
    # Process data for each model using the create_timepoint_ega_dataframe function
    model1_timepoint_table = create_timepoint_ega_dataframe(model1, glycemic_region)
    model2_timepoint_table = create_timepoint_ega_dataframe(model2, glycemic_region)

    # Plot the Erroneous Prediction % for both models
    plt.figure(figsize=(10, 4))

    model1_color = (173 / 255, 29 / 255, 30 / 255)  # Without Feature Enhancement
    model2_color = (110 / 255, 180 / 255, 186 / 255)  # With Feature Enhancement


    plt.plot(model1_timepoint_table.index[1:], model1_timepoint_table['EP_percent'][1:], 
             label='Population JPFormer', marker='o', linestyle='-', markersize=4, 
             color=model1_color)
    plt.plot(model2_timepoint_table.index[1:], model2_timepoint_table['EP_percent'][1:], 
             label='Personalised JPFormer', marker='o', linestyle='-', markersize=4, 
             color=model2_color)

    # Add titles and labels

    plt.title(f'EP%  by Prediction Horizon', 
              fontsize=16, fontweight='bold', ha='center', pad=20)
    plt.xlabel('Prediction Horizon (mins)', fontsize=14, labelpad=5, ha='center')
    plt.ylabel('EP Percent', fontsize=14, labelpad=5)
    plt.xticks(range(0, 121, 30), fontsize=14)
    plt.yticks(fontsize=14)

    # Format legend
    legend = plt.legend(title='Model', fontsize=14, loc='upper center', bbox_to_anchor=(0.5, -0.2), 
                        ncol=2, frameon=False)

    legend.get_title().set_fontsize(14)

    # Add grid and styling
    plt.grid(axis='y', linestyle='-', alpha=0.6)
    plt.ylim(0, 50)  
    sns.despine()  # Remove top and right borders
    plt.tight_layout()
    plt.show()


In [ ]:
# Call the function with the appropriate arguments'
return_ep_ph_chart(model1=aggregate_base_df, model2=aggregate_fine_tuned_df, glycemic_region='overall')

return_ep_ph_chart(model1=aggregate_base_df, model2=aggregate_fine_tuned_df, glycemic_region='hypo')

return_ep_ph_chart(model1=aggregate_base_df, model2=aggregate_fine_tuned_df, glycemic_region='eu')

return_ep_ph_chart(model1=aggregate_base_df, model2=aggregate_fine_tuned_df, glycemic_region='hyper')

In [ ]:
def return_ap_ph_chart(model1, model2, glycemic_region):
    # Process data for each model using the create_timepoint_ega_dataframe function
    model1_timepoint_table = create_timepoint_ega_dataframe(model1, glycemic_region)
    model2_timepoint_table = create_timepoint_ega_dataframe(model2, glycemic_region)

    # Plot the Erroneous Prediction % for both models
    plt.figure(figsize=(10, 4))

    model1_color = (173 / 255, 29 / 255, 30 / 255)  # Without Feature Enhancement
    model2_color = (110 / 255, 180 / 255, 186 / 255)  # With Feature Enhancement



    plt.plot(model1_timepoint_table.index[1:], model1_timepoint_table['AP_percent'][1:], 
             label='Population JPFormer', marker='o', linestyle='-', markersize=4, 
             color=model1_color)
    plt.plot(model2_timepoint_table.index[1:], model2_timepoint_table['AP_percent'][1:], 
             label='Personalised JPFormer', marker='o', linestyle='-', markersize=4, 
             color=model2_color)

    # Add titles and labels

    plt.title(f'AP%  by Prediction Horizon', 
              fontsize=16, fontweight='bold', ha='center', pad=20)
    plt.xlabel('Prediction Horizon (mins)', fontsize=14, labelpad=5, ha='center')
    plt.ylabel('AP Percent', fontsize=14, labelpad=5)
    plt.xticks(range(0, 121, 30), fontsize=14)
    plt.yticks(fontsize=14)

    # Format legend
    legend = plt.legend(title='Model', fontsize=14, loc='upper center', bbox_to_anchor=(0.5, -0.2), 
                        ncol=2, frameon=False)

    legend.get_title().set_fontsize(14)

    # Add grid and styling
    plt.grid(axis='y', linestyle='-', alpha=0.6)
    plt.ylim(50, 100)  
    sns.despine()  # Remove top and right borders
    plt.tight_layout()
    plt.show()


In [ ]:
# Call the function with the appropriate arguments'
return_ap_ph_chart(model1=aggregate_base_df, model2=aggregate_fine_tuned_df, glycemic_region='overall')

return_ap_ph_chart(model1=aggregate_base_df, model2=aggregate_fine_tuned_df, glycemic_region='hypo')

return_ap_ph_chart(model1=aggregate_base_df, model2=aggregate_fine_tuned_df, glycemic_region='eu')

return_ap_ph_chart(model1=aggregate_base_df, model2=aggregate_fine_tuned_df, glycemic_region='hyper')

In [ ]:
def return_timepoint_summary(no_fe_model, fe_model, glycemic_region):

    timepoints = [30, 60, 90, 120]

    # Create a dictionary to store DataFrames for each timepoint
    timepoint_dfs = {}

    # For each timepoint, create a DataFrame with rows from both models
    for t in timepoints:
        # Create an empty DataFrame
        timepoint_df = pd.DataFrame(columns=['Model', 'AP_percent', 'BP_percent', 'EP_percent', 
                                            'AP_count', 'BP_count', 'EP_count', 'Total_count'])
        
            # Process data for each model using the create_timepoint_ega_dataframe function
        model1_timepoint_table = create_timepoint_ega_dataframe(no_fe_model, glycemic_region)
        model2_timepoint_table = create_timepoint_ega_dataframe(fe_model, glycemic_region)

        # Get data from both models for this timepoint
        no_fe_data = model1_timepoint_table.loc[t].copy()
        fe_data = model2_timepoint_table.loc[t].copy()
        
        # Create rows with model identifiers
        no_fe_row = pd.DataFrame({
            'Model': ['Population JPFormer'],
            'AP_percent': [no_fe_data['AP_percent']],
            'BP_percent': [no_fe_data['BP_percent']],
            'EP_percent': [no_fe_data['EP_percent']],
            'AP_count': [no_fe_data['AP_count']],
            'BP_count': [no_fe_data['BP_count']],
            'EP_count': [no_fe_data['EP_count']],
            'Total_count': [no_fe_data['Total_count']],
            'AP:EP': [no_fe_data['AP:EP']]
        })
        
        fe_row = pd.DataFrame({
            'Model': ['Personalised JPFormer'],
            'AP_percent': [fe_data['AP_percent']],
            'BP_percent': [fe_data['BP_percent']],
            'EP_percent': [fe_data['EP_percent']],
            'AP_count': [fe_data['AP_count']],
            'BP_count': [fe_data['BP_count']],
            'EP_count': [fe_data['EP_count']],
            'Total_count': [fe_data['Total_count']],
            'AP:EP': [fe_data['AP:EP']]
        })
        
        # Combine the two rows
        timepoint_df = pd.concat([no_fe_row, fe_row])
        
        # Store in dictionary
        timepoint_dfs[t] = timepoint_df

    # Create a figure to visualize EP percentages across timepoints
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    axs = axs.flatten()

    # Define custom colors for the models
    model1_color = (173 / 255, 29 / 255, 30 / 255)  # Without Feature Enhancement
    model2_color = (110 / 255, 180 / 255, 186 / 255)  # With Feature Enhancement
    palette = [model1_color, model2_color]

    # Plot bar charts for each timepoint
    for i, t in enumerate(timepoints):
        sns.barplot(
            x='Model', 
            y='EP_percent', 
            data=timepoint_dfs[t], 
            ax=axs[i], 
            palette=palette,
            hue='Model',
            legend=False
        )
        axs[i].set_title(f'Prediction Horizon: {t} mins', fontsize=16, fontweight='bold', pad=20)
        axs[i].set_ylabel('EP Precentage', fontsize=14)
        axs[i].set_xlabel('')
        axs[i].set_ylim(0, 50)  # Set consistent y-axis limits
        
        # Add data labels
        for p in axs[i].patches:
            axs[i].annotate(f"{p.get_height():.1f}%", 
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='bottom', fontsize=14)
        
        # Set x-tick labels font size to 8
        axs[i].tick_params(axis='x', labelsize=12)
        
        # Remove top and right borders
        sns.despine(ax=axs[i])
    glycemic_region = glycemic_region.capitalize()


    # Adjust layout for better spacing
    plt.tight_layout()
    plt.subplots_adjust(top=0.85, hspace=0.4)  # Increased space between title and plots
    plt.show()

    # Display the comparison tables
    for t in timepoints:
        print(f"\nPrediction Horizon: {t} minutes")
        display(timepoint_dfs[t])




In [ ]:
return_timepoint_summary(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region='overall')

return_timepoint_summary(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region='hypo')

return_timepoint_summary(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region='eu')

return_timepoint_summary(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region='hyper')


In [ ]:
def return_ap_timepoint_summary(no_fe_model, fe_model, glycemic_region):

    timepoints = [30, 60, 90, 120]

    # Create a dictionary to store DataFrames for each timepoint
    timepoint_dfs = {}

    # For each timepoint, create a DataFrame with rows from both models
    for t in timepoints:
        # Create an empty DataFrame
        timepoint_df = pd.DataFrame(columns=['Model', 'AP_percent', 'BP_percent', 'EP_percent', 
                                            'AP_count', 'BP_count', 'EP_count', 'Total_count'])
        
        # Process data for each model using the create_timepoint_ega_dataframe function
        model1_timepoint_table = create_timepoint_ega_dataframe(no_fe_model, glycemic_region)
        model2_timepoint_table = create_timepoint_ega_dataframe(fe_model, glycemic_region)

        # Get data from both models for this timepoint
        no_fe_data = model1_timepoint_table.loc[t].copy()
        fe_data = model2_timepoint_table.loc[t].copy()
        
        # Create rows with model identifiers
        no_fe_row = pd.DataFrame({
            'Model': ['base model'],
            'AP_percent': [no_fe_data['AP_percent']],
            'BP_percent': [no_fe_data['BP_percent']],
            'EP_percent': [no_fe_data['EP_percent']],
            'AP_count': [no_fe_data['AP_count']],
            'BP_count': [no_fe_data['BP_count']],
            'EP_count': [no_fe_data['EP_count']],
            'Total_count': [no_fe_data['Total_count']],
            'AP:EP': [no_fe_data['AP:EP']]
        })
        
        fe_row = pd.DataFrame({
            'Model': ['fine-tuned models'],
            'AP_percent': [fe_data['AP_percent']],
            'BP_percent': [fe_data['BP_percent']],
            'EP_percent': [fe_data['EP_percent']],
            'AP_count': [fe_data['AP_count']],
            'BP_count': [fe_data['BP_count']],
            'EP_count': [fe_data['EP_count']],
            'Total_count': [fe_data['Total_count']],
            'AP:EP': [fe_data['AP:EP']]
        })
        
        # Combine the two rows
        timepoint_df = pd.concat([no_fe_row, fe_row])
        
        # Store in dictionary
        timepoint_dfs[t] = timepoint_df

    # Create a figure to visualize AP percentages across timepoints
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    axs = axs.flatten()

    # Define custom colors for the models
    model1_color = (173 / 255, 29 / 255, 30 / 255)  # Without Feature Enhancement
    model2_color = (110 / 255, 180 / 255, 186 / 255)  # With Feature Enhancement
    palette = [model1_color, model2_color]

    # Plot bar charts for each timepoint
    for i, t in enumerate(timepoints):
        sns.barplot(
            x='Model', 
            y='AP_percent',  # Changed from EP_percent to AP_percent
            data=timepoint_dfs[t], 
            ax=axs[i], 
            palette=palette,
            hue='Model',
            legend=False
        )
        axs[i].set_title(f'Prediction Horizon: {t} mins', fontsize=16, fontweight='bold', pad=20)
        axs[i].set_ylabel('AP Percentage', fontsize=14)  # Updated label
        axs[i].set_xlabel('')
        axs[i].set_ylim(50, 100)  # Adjusted y-axis limits for AP percentages
        
        # Add data labels
        for p in axs[i].patches:
            axs[i].annotate(f"{p.get_height():.1f}%", 
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='bottom', fontsize=14)
        
        # Set x-tick labels font size to 8
        axs[i].tick_params(axis='x', labelsize=14)
        
        # Remove top and right borders
        sns.despine(ax=axs[i])
    glycemic_region = glycemic_region.capitalize()

    # Adjust layout for better spacing
    plt.tight_layout()
    plt.subplots_adjust(top=0.85, hspace=0.4)  # Increased space between title and plots
    plt.show()

    # Display the comparison tables
    for t in timepoints:
        print(f"\nPrediction Horizon: {t} minutes")
        display(timepoint_dfs[t])


In [ ]:
return_ap_timepoint_summary(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region='overall')
return_ap_timepoint_summary(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region='hypo')
return_ap_timepoint_summary(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region='eu')
return_ap_timepoint_summary(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region='hyper')

### **4.1. Statistical Analysis**

In [ ]:
from IPython.display import display

# Function to compute Cramér's V
def cramers_v(chi2, n, contingency_table):
    """Computes Cramér's V effect size from the chi-square test."""
    min_dim = min(np.shape(contingency_table)) - 1  # Min(row-1, col-1)
    return np.sqrt(chi2 / (n * min_dim)) if min_dim > 0 else np.nan

# Function to interpret Cramér's V effect size
def interpret_cramers_v(v_value):
    """Provides qualitative interpretation of Cramér's V."""
    if pd.isna(v_value):  
        return "Not Enough Data"
    elif v_value < 0.1:
        return "Negligible Effect"
    elif v_value < 0.3:
        return "Small Effect"
    elif v_value < 0.5:
        return "Medium Effect"
    else:
        return "Large Effect"

# Chi-square analysis function
def return_chi_square_analysis(no_fe_model, fe_model, glycemic_region, model1_name="Model w/o FE", model2_name="Model w/ FE"):
    glycemic_region = glycemic_region.lower()
    data_dist_chi_square_results = []

    print(f"\n Running Chi-Square test for {model1_name} vs {model2_name} ({glycemic_region.capitalize()}glycaemic Range Performance):\n")

    # Copy data to avoid modifying the originals
    df1 = no_fe_model.copy()
    df2 = fe_model.copy()

    # Filter based on glycemic region
    if glycemic_region == 'hypo':
        df1 = df1[df1['glycemic_region'] == 'hypo']
        df2 = df2[df2['glycemic_region'] == 'hypo']
    elif glycemic_region == 'hyper':
        df1 = df1[df1['glycemic_region'] == 'hyper']
        df2 = df2[df2['glycemic_region'] == 'hyper']
    elif glycemic_region == 'eu':
        df1 = df1[df1['glycemic_region'] == 'eu']
        df2 = df2[df2['glycemic_region'] == 'eu']
    else:
        pass  # Use all data

    # Calculate class counts
    df1_counts = df1['CG_EGA_Class'].value_counts()
    df2_counts = df2['CG_EGA_Class'].value_counts()
    
    # Totals
    df1_total = df1_counts.sum()
    df2_total = df2_counts.sum()

    # Contingency table
    contingency_table = [
        [df1_counts.get('AP', 0), df1_counts.get('BE', 0), df1_counts.get('EP', 0)],
        [df2_counts.get('AP', 0), df2_counts.get('BE', 0), df2_counts.get('EP', 0)],
    ]

    # Percentages
    df1_percent = {
        'AP': (df1_counts.get('AP', 0) / df1_total) * 100 if df1_total else 0,
        'BE': (df1_counts.get('BE', 0) / df1_total) * 100 if df1_total else 0,
        'EP': (df1_counts.get('EP', 0) / df1_total) * 100 if df1_total else 0
    }
    df2_percent = {
        'AP': (df2_counts.get('AP', 0) / df2_total) * 100 if df2_total else 0,
        'BE': (df2_counts.get('BE', 0) / df2_total) * 100 if df2_total else 0,
        'EP': (df2_counts.get('EP', 0) / df2_total) * 100 if df2_total else 0
    }

    # Chi-square test
    chi2, p_value, _, _ = stats.chi2_contingency(contingency_table)

    # Cramér's V
    n = np.sum(contingency_table)
    cramers_v_value = cramers_v(chi2, n, contingency_table)
    cramers_v_interpretation = interpret_cramers_v(cramers_v_value)

    # Store result
    data_dist_chi_square_results.append({
        "Model 1": f"{model1_name} ({glycemic_region.capitalize()})",
        "Model 2": f"{model2_name} ({glycemic_region.capitalize()})",
        "Chi2 Statistic": chi2,
        "p-value": p_value,
        "Significant": p_value < 0.05,
        "Cramér's V": cramers_v_value,
        "Cramér's V Interpretation": cramers_v_interpretation
    })

    # Display results
    print(f"\n  χ² = {chi2:.3f}, p = {p_value:.5f}, "
          f"{'Significant' if p_value < 0.05 else 'Not Significant'} | "
          f"Cramér's V = {cramers_v_value:.3f} ({cramers_v_interpretation})\n")

    # Contingency table (readable)
    table_data = pd.DataFrame({
        "AP (Accurate Predictions)": [f"{df1_counts.get('AP', 0)} ({df1_percent['AP']:.2f}%)", 
                                     f"{df2_counts.get('AP', 0)} ({df2_percent['AP']:.2f}%)"],
        "BE (Benign Errors)": [f"{df1_counts.get('BE', 0)} ({df1_percent['BE']:.2f}%)", 
                              f"{df2_counts.get('BE', 0)} ({df2_percent['BE']:.2f}%)"],
        "EP (Erroneous Predictions)": [f"{df1_counts.get('EP', 0)} ({df1_percent['EP']:.2f}%)", 
                                      f"{df2_counts.get('EP', 0)} ({df2_percent['EP']:.2f}%)"]
    }, index=[model1_name, model2_name])

    print("\nContingency Table:")
    display(table_data)

    return pd.DataFrame(data_dist_chi_square_results)

In [ ]:
overall_chi_square_results = return_chi_square_analysis(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region='overall', model1_name="Base model", model2_name="Fine-tuned models")
hypo_chi_square_results = return_chi_square_analysis(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region='hypo', model1_name="Base model", model2_name="Fine-tuned models")
eu_chi_square_results = return_chi_square_analysis(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region='eu', model1_name="Base model", model2_name="Fine-tuned models")
hyper_chi_square_results = return_chi_square_analysis(aggregate_base_df, aggregate_fine_tuned_df, glycemic_region='hyper', model1_name="Base model", model2_name="Fine-tuned models")

In [ ]:
# Extract rows for specific prediction horizons (timepoints)
population_30_df = aggregate_base_df[aggregate_base_df['timepoint'] == 5]
population_60_df = aggregate_base_df[aggregate_base_df['timepoint'] == 11]
population_90_df = aggregate_base_df[aggregate_base_df['timepoint'] == 17]
population_120_df = aggregate_base_df[aggregate_base_df['timepoint'] == 23]

# Do the same for the fine-tuned model
personalized_30_df = aggregate_fine_tuned_df[aggregate_fine_tuned_df['timepoint'] == 5]
personalized_60_df = aggregate_fine_tuned_df[aggregate_fine_tuned_df['timepoint'] == 11]
personalized_90_df = aggregate_fine_tuned_df[aggregate_fine_tuned_df['timepoint'] == 17]
personalized_120_df = aggregate_fine_tuned_df[aggregate_fine_tuned_df['timepoint'] == 23]

In [ ]:
return_chi_square_analysis(population_30_df, personalized_30_df, glycemic_region='hypo', model1_name="Population JPFormer 30", model2_name="Personalised JPFormer 30")
return_chi_square_analysis(population_60_df, personalized_60_df, glycemic_region='hypo', model1_name="Population JPFormer 60", model2_name="Personalised JPFormer 60")
return_chi_square_analysis(population_90_df, personalized_90_df, glycemic_region='hypo', model1_name="Population JPFormer 90", model2_name="Personalised JPFormer 90")
return_chi_square_analysis(population_120_df, personalized_120_df, glycemic_region='hypo', model1_name="Population JPFormer 120", model2_name="Personalised JPFormer 120")

## **Individual Level Analysis**

### RMSE and MAPE

In [ ]:
ptid_base_rmse_dict = {}

ptid_fine_tuned_rmse_dict = {}



for ptid, df in base_overall_dict.items():
    base_rmse = calculate_rmse(df['true_glucose'], df['predicted_glucose'])
    ptid_base_rmse_dict[ptid] = base_rmse

for ptid, df in fine_tuned_dict.items():
    fine_tuned_rmse = calculate_rmse(df['true_glucose'], df['predicted_glucose'])
    ptid_fine_tuned_rmse_dict[ptid] = fine_tuned_rmse



In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt

# Create a figure with a 3x4 grid layout
fig, axs = plt.subplots(3, 4, figsize=(16, 12))
axs = axs.flatten()  # Convert to 1D array for easier indexing

# Define custom colors for the models
model1_color = (173 / 255, 29 / 255, 30 / 255)  # Base model
model2_color = (110 / 255, 180 / 255, 186 / 255)  # Fine-tuned model
palette = [model1_color, model2_color]

# Loop through each patient ID and create a bar chart in each subplot
for i, ptid in enumerate(ptid_list):
    # Create a DataFrame for this patient's RMSE values
    df = pd.DataFrame({
        'Model': ['Base Model', 'Fine-Tuned Model'],
        'RMSE': [ptid_base_rmse_dict[ptid], ptid_fine_tuned_rmse_dict[ptid]]
    })
    
    # Create bar chart in the corresponding subplot
    sns.barplot(x='Model', y='RMSE', data=df, ax=axs[i], palette=palette)
    
    # Set title and styling for the subplot
    axs[i].set_title(f'Patient {ptid}', fontsize=12, fontweight='bold')
    axs[i].set_ylabel('RMSE (mg/dL)', fontsize=10)
    axs[i].set_xlabel('')
    
    # Add data labels
    for p in axs[i].patches:
        axs[i].annotate(f"{p.get_height():.1f}", 
                      (p.get_x() + p.get_width() / 2., p.get_height()),
                      ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    # Remove top and right borders
    sns.despine(ax=axs[i])
    
    # Set y-axis limits based on the data range
    max_value = max(ptid_base_rmse_dict[ptid], ptid_fine_tuned_rmse_dict[ptid])
    axs[i].set_ylim(0, 50)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.suptitle('RMSE Comparison by Person ID: Base Model vs Fine-Tuned Model', 
             fontsize=16, fontweight='bold', y=1.02)
plt.subplots_adjust(top=0.95)
plt.show()

In [ ]:
ptid_base_mape_dict = {}
ptid_fine_tuned_mape_dict = {}

for ptid, df in base_overall_dict.items():
    base_mape = mape(df['true_glucose'], df['predicted_glucose'])
    ptid_base_mape_dict[ptid] = base_mape

for ptid, df in fine_tuned_dict.items():
    fine_tuned_mape = mape(df['true_glucose'], df['predicted_glucose'])
    ptid_fine_tuned_mape_dict[ptid] = fine_tuned_mape

In [ ]:

# Create a figure with a 3x4 grid layout
fig, axs = plt.subplots(3, 4, figsize=(16, 12))
axs = axs.flatten()  # Convert to 1D array for easier indexing

# Define custom colors for the models
model1_color = (173 / 255, 29 / 255, 30 / 255)  # Base model
model2_color = (110 / 255, 180 / 255, 186 / 255)  # Fine-tuned model
palette = [model1_color, model2_color]

# Loop through each patient ID and create a bar chart in each subplot
for i, ptid in enumerate(ptid_list):
    # Create a DataFrame for this patient's RMSE values
    df = pd.DataFrame({
        'Model': ['Base Model', 'Fine-Tuned Model'],
        'MAPE': [ptid_base_mape_dict[ptid], ptid_fine_tuned_mape_dict[ptid]]
    })
    
    # Create bar chart in the corresponding subplot
    sns.barplot(x='Model', y='MAPE', data=df, ax=axs[i], palette=palette)
    
    # Set title and styling for the subplot
    axs[i].set_title(f'Patient {ptid}', fontsize=12, fontweight='bold')
    axs[i].set_ylabel('RMSE (mg/dL)', fontsize=10)
    axs[i].set_xlabel('')
    
    # Add data labels
    for p in axs[i].patches:
        axs[i].annotate(f"{p.get_height():.1f}", 
                      (p.get_x() + p.get_width() / 2., p.get_height()),
                      ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    # Remove top and right borders
    sns.despine(ax=axs[i])
    
    axs[i].set_ylim(0, 50)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.suptitle('MAPE Comparison by Person ID: Base Model vs Fine-Tuned Model', 
             fontsize=16, fontweight='bold', y=1.02)
plt.subplots_adjust(top=0.95)
plt.show()

In [ ]:
ttest_results_dict = {}
for ptid in ptid_list:
    ttest_results = return_ttests(base_overall_dict[ptid], fine_tuned_dict[ptid], glycemic_region="overall", model1_name="Base Model", model2_name="Fine-tuned Model")
    ttest_results_dict[ptid] = ttest_results



In [ ]:
# create single dataframe to store all t-test results
ttest_results_rmse_df = pd.DataFrame(columns=["Patient ID", "Base RMSE", "Fine-tuned RMSE", "t-statistic", "p-value", "Significant"])
for ptid, results in ttest_results_dict.items():
    # Extract RMSE row from the t-test results
    rmse_row = results.loc[0]
    new_row = pd.DataFrame({
        "Patient ID": [ptid],
        "Base RMSE": [ptid_base_rmse_dict[ptid]],
        "Fine-tuned RMSE": [ptid_fine_tuned_rmse_dict[ptid]],
        "t-statistic": [rmse_row["t-statistic"]],
        "p-value": [rmse_row["p-value"]],
        "Significant": [rmse_row["p-value"] < 0.05]
    })
    ttest_results_rmse_df = pd.concat([ttest_results_rmse_df, new_row], ignore_index=True)

# Format RMSE values to 2 decimal places
ttest_results_rmse_df["Base RMSE"] = ttest_results_rmse_df["Base RMSE"].round(2)
ttest_results_rmse_df["Fine-tuned RMSE"] = ttest_results_rmse_df["Fine-tuned RMSE"].round(2)
ttest_results_rmse_df["t-statistic"] = ttest_results_rmse_df["t-statistic"].round(3)
ttest_results_rmse_df["p-value"] = ttest_results_rmse_df["p-value"].map(lambda x: f"{x:.4f}")

ttest_results_rmse_df

In [ ]:
ttest_results_mape_df = pd.DataFrame(columns=["Patient ID", "Base MAPE", "Fine-tuned MAPE", "t-statistic", "p-value", "Significant"])
for ptid, results in ttest_results_dict.items():
    # Extract MAPE row from the t-test results
    mape_row = results.loc[1]
    new_row = pd.DataFrame({
        "Patient ID": [ptid],
        "Base MAPE": [ptid_base_mape_dict[ptid]],
        "Fine-tuned MAPE": [ptid_fine_tuned_mape_dict[ptid]],
        "t-statistic": [mape_row["t-statistic"]],
        "p-value": [mape_row["p-value"]],
        "Significant": [mape_row["p-value"] < 0.05]
    })
    ttest_results_mape_df = pd.concat([ttest_results_mape_df, new_row], ignore_index=True)

# Format MAPE values to 2 decimal places
ttest_results_mape_df["Base MAPE"] = ttest_results_mape_df["Base MAPE"].round(2)
ttest_results_mape_df["Fine-tuned MAPE"] = ttest_results_mape_df["Fine-tuned MAPE"].round(2)
ttest_results_mape_df["t-statistic"] = ttest_results_mape_df["t-statistic"].round(3)
ttest_results_mape_df["p-value"] = ttest_results_mape_df["p-value"].map(lambda x: f"{x:.4f}")

ttest_results_mape_df

In [ ]:
# Create a figure with a 1x2 grid layout
fig, axs = plt.subplots(1, 2, figsize=(10, 10))

# RMSE Difference Chart
df_rmse_diff = pd.DataFrame({
    'Patient ID': [f"Patient {ptid}" for ptid in ptid_rmse_diff.keys()],
    'RMSE Difference': [-diff for diff in ptid_rmse_diff.values()]  # Negate to make positive = base better
})
df_rmse_diff = df_rmse_diff.sort_values('RMSE Difference')
colors_rmse = ['#AEB4B8'] * len(df_rmse_diff)
for i, val in enumerate(df_rmse_diff['RMSE Difference']):
    colors_rmse[i] = model2_color if val > 0 else model1_color
bars_rmse = axs[0].barh(df_rmse_diff['Patient ID'], df_rmse_diff['RMSE Difference'], color=colors_rmse)
axs[0].axvline(0, color='black', linestyle='-', linewidth=1)

axs[0].set_xlabel('RMSE (mg/dL)', fontsize=14)
axs[0].set_xlim(-6, 6)
# Add data labels with asterisks for significant values
for i, bar in enumerate(bars_rmse):
    width = bar.get_width()
    label_x_pos = width + 0.1 if width > 0 else width - 0.1
    
    # Get the patient ID number
    ptid = int(df_rmse_diff.iloc[i]['Patient ID'].split(' ')[1])
    
    # Check if this patient has a significant p-value
    is_significant = ttest_results_rmse_df.loc[ttest_results_rmse_df['Patient ID'] == ptid, 'Significant'].values[0]
    
    # Add an asterisk if significant
    suffix = "*" if is_significant == True else ""
    
    axs[0].text(label_x_pos, bar.get_y() + bar.get_height()/2, f'{abs(width):.2f}{suffix}',
                va='center', ha='left' if width > 0 else 'right', fontsize=12)
sns.despine(ax=axs[0])

# MAPE Difference Chart
df_mape_diff = pd.DataFrame({
    'Patient ID': [f"Patient {ptid}" for ptid in ptid_mape_diff.keys()],
    'MAPE Difference': [-diff for diff in ptid_mape_diff.values()]  # Negate to make positive = base better
})
df_mape_diff = df_mape_diff.sort_values('MAPE Difference')
colors_mape = ['#AEB4B8'] * len(df_mape_diff)
for i, val in enumerate(df_mape_diff['MAPE Difference']):
    colors_mape[i] = model2_color if val > 0 else model1_color
bars_mape = axs[1].barh(df_mape_diff['Patient ID'], df_mape_diff['MAPE Difference'], color=colors_mape)
axs[1].axvline(0, color='black', linestyle='-', linewidth=1)

axs[1].set_xlabel('MAPE ', fontsize=14)
axs[1].set_xlim(-6, 6)
# Add data labels with asterisks for significant values
for i, bar in enumerate(bars_mape):
    width = bar.get_width()
    label_x_pos = width + 0.1 if width > 0 else width - 0.1
    
    # Get the patient ID number
    ptid = int(df_mape_diff.iloc[i]['Patient ID'].split(' ')[1])
    
    # Get p-value and check if it's significant
    p_value = float(ttest_results_mape_df.loc[ttest_results_mape_df['Patient ID'] == ptid, 'p-value'])
    is_significant = p_value < 0.05
    
    # Add an asterisk if significant
    suffix = "*" if is_significant else ""
    
    axs[1].text(label_x_pos, bar.get_y() + bar.get_height()/2, f'{abs(width):.2f}{suffix}',
                va='center', ha='left' if width > 0 else 'right', fontsize=12)
sns.despine(ax=axs[1])

for ax in axs:
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.text(xlim[0]+0.25, ylim[0] - 0.8, 'Population model\n better', 
            color=model1_color, fontsize=12, fontweight='bold', ha='left', va='top')
    ax.text(xlim[1]-0.25, ylim[0] - 0.8, 'Personalised model\n better', 
            color=model2_color, fontsize=12, fontweight='bold', ha='right', va='top')

plt.suptitle('Overall RMSE and MAPE Difference: Population vs Personalised JPFormer',
             fontsize=16, fontweight='bold', y=1.02)
# # Add a note explaining the asterisk
# fig.text(0.5, 0.01, '* indicates statistically significant difference (p < 0.05)',
#          ha='center', fontsize=12, fontstyle='italic')
# plt.subplots_adjust(top=0.95, bottom=0.05)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


### CG-EGA

In [ ]:
updated_base_df = {}
updated_fine_tuned_df = {}


for ptid in ptid_list:
    base_df = base_overall_dict[ptid]

    base_df['AP'] = np.where(base_df['CG_EGA_Class'] == 'AP', 1, 0)
    base_df['BE'] = np.where(base_df['CG_EGA_Class'] == 'BE', 1, 0)
    base_df['EP'] = np.where(base_df['CG_EGA_Class'] == 'EP', 1, 0)
    
    updated_base_df[ptid] = base_df


    fine_tuned_df = fine_tuned_dict[ptid]
    fine_tuned_df['AP'] = np.where(fine_tuned_df['CG_EGA_Class'] == 'AP', 1, 0)
    fine_tuned_df['BE'] = np.where(fine_tuned_df['CG_EGA_Class'] == 'BE', 1, 0)
    fine_tuned_df['EP'] = np.where(fine_tuned_df['CG_EGA_Class'] == 'EP', 1, 0)
    
    updated_fine_tuned_df[ptid] = fine_tuned_df


In [ ]:
# run chi2 test for each patient id for overall model performacne

overall_chi2_results_dict = {}
hypo_chi2_results_dict  = {}
eu_chi2_results_dict  = {}
hyper_chi2_results_dict  = {}

for ptid in ptid_list:

    overall_chi2_results_dict[ptid] = return_chi_square_analysis(updated_base_df[ptid], updated_fine_tuned_df[ptid], glycemic_region='overall', model1_name='Population JPFormer', model2_name='Personalised JPFormer')
    

    hypo_chi2_results_dict[ptid] = return_chi_square_analysis(updated_base_df[ptid], updated_fine_tuned_df[ptid], glycemic_region='hypo', model1_name='Population JPFormer', model2_name='Personalised JPFormer')


    eu_chi2_results_dict[ptid] = return_chi_square_analysis(updated_base_df[ptid], updated_fine_tuned_df[ptid], glycemic_region='eu', model1_name='Population JPFormer', model2_name='Personalised JPFormer')


    hyper_chi2_results_dict[ptid] = return_chi_square_analysis(updated_base_df[ptid], updated_fine_tuned_df[ptid], glycemic_region='hyper', model1_name='Population JPFormer', model2_name='Personalised JPFormer')

In [ ]:
# Create summary dataframes for each glycemic region
overall_chi2_summary = pd.DataFrame(columns=["Patient ID", "Chi2", "p-value", "Significant"])
hypo_chi2_summary = pd.DataFrame(columns=["Patient ID", "Chi2", "p-value",  "Significant"])
eu_chi2_summary = pd.DataFrame(columns=["Patient ID", "Chi2", "p-value", "Significant"])
hyper_chi2_summary = pd.DataFrame(columns=["Patient ID", "Chi2", "p-value", "Significant"])

# Populate the dataframes with results from each patient
for ptid in ptid_list:
    # Overall results
    overall_row = pd.DataFrame({
        "Patient ID": [ptid],
        "Chi2": [overall_chi2_results_dict[ptid]["Chi2 Statistic"].iloc[0]],
        "p-value": [overall_chi2_results_dict[ptid]["p-value"].iloc[0]],
        "Significant": [overall_chi2_results_dict[ptid]["p-value"].iloc[0] < 0.05]

    })
    overall_chi2_summary = pd.concat([overall_chi2_summary, overall_row], ignore_index=True)
    
    # Hypo results
    hypo_row = pd.DataFrame({
        "Patient ID": [ptid],
        "Chi2": [hypo_chi2_results_dict[ptid]["Chi2 Statistic"].iloc[0]],
        "p-value": [hypo_chi2_results_dict[ptid]["p-value"].iloc[0]],
        "Significant": [hypo_chi2_results_dict[ptid]["p-value"].iloc[0] < 0.05]

    })
    hypo_chi2_summary = pd.concat([hypo_chi2_summary, hypo_row], ignore_index=True)
    
    # Eu results
    eu_row = pd.DataFrame({
        "Patient ID": [ptid],
        "Chi2": [eu_chi2_results_dict[ptid]["Chi2 Statistic"].iloc[0]],
        "p-value": [eu_chi2_results_dict[ptid]["p-value"].iloc[0]],
        "Significant": [eu_chi2_results_dict[ptid]["p-value"].iloc[0] < 0.05]

    })
    eu_chi2_summary = pd.concat([eu_chi2_summary, eu_row], ignore_index=True)
    
    # Hyper results
    hyper_row = pd.DataFrame({
        "Patient ID": [ptid],
        "Chi2": [hyper_chi2_results_dict[ptid]["Chi2 Statistic"].iloc[0]],
        "p-value": [hyper_chi2_results_dict[ptid]["p-value"].iloc[0]],
        "Significant": [hyper_chi2_results_dict[ptid]["p-value"].iloc[0] < 0.05]

    })
    hyper_chi2_summary = pd.concat([hyper_chi2_summary, hyper_row], ignore_index=True)

# Format the dataframes
for df in [overall_chi2_summary, hypo_chi2_summary, eu_chi2_summary, hyper_chi2_summary]:
    df["Chi2"] = df["Chi2"].round(3)
    df["p-value"] = df["p-value"].map(lambda x: f"{x:.4f}")

    df["Patient ID"] = df["Patient ID"].astype(int)

# Display the summaries
print("Overall Chi-Square Test Results:")
display(overall_chi2_summary)

print("\nHypoglycemia Chi-Square Test Results:")
display(hypo_chi2_summary)

print("\nEuglycemia Chi-Square Test Results:")
display(eu_chi2_summary)

print("\nHyperglycemia Chi-Square Test Results:")
display(hyper_chi2_summary)



In [ ]:
for ptid in ptid_list:
    print(f"Patient ID: {ptid}")

    # Define custom colors for the models
    model1_color = (173 / 255, 29 / 255, 30 / 255)  # Without Feature Enhancement
    model2_color = (110 / 255, 180 / 255, 186 / 255)  # With Feature Enhancement
    palette = [model1_color, model2_color]

    # Create summary tables for overall, hypo, eu, and hyper glycaemic regions
    regions = ['overall', 'hypo', 'eu', 'hyper']
    summary_tables = {}

    # Get the dataframes for the current patient ID
    base_df = updated_base_df[ptid]
    fine_tuned_df = updated_fine_tuned_df[ptid]
    
    for region in regions:
        # Initialize empty DataFrame with specific dtypes to avoid warning
        summary_df = pd.DataFrame({
            'Model': pd.Series(dtype='object'),
            'AP': pd.Series(dtype='int64'), 
            'BE': pd.Series(dtype='int64'), 
            'EP': pd.Series(dtype='int64'), 
            'Count': pd.Series(dtype='int64'),
            'AP_pct': pd.Series(dtype='float64'), 
            'BE_pct': pd.Series(dtype='float64'), 
            'EP_pct': pd.Series(dtype='float64')
        })
        
        # Process each model
        for model, df in zip(['base model', 'fine-tuned models'], 
                            [base_df, fine_tuned_df]):
            
            # Filter for region if not overall
            if region != 'overall':
                region_df = df[df['glycemic_region'] == region]
            else:
                region_df = df
                
            # Calculate counts
            ap_count = region_df['AP'].sum()
            be_count = region_df['BE'].sum()
            ep_count = region_df['EP'].sum()
            total_count = len(region_df)
            
            # Create a new row as a dictionary and append it to the DataFrame
            new_row = {
                'Model': model, 
                'AP': ap_count, 
                'BE': be_count, 
                'EP': ep_count, 
                'Count': total_count,
                'AP:EP': ap_count / ep_count if ep_count != 0 else np.nan,
                'AP_pct': ap_count / total_count * 100, 
                'BE_pct': be_count / total_count * 100, 
                'EP_pct': ep_count / total_count * 100
            }
            summary_df = pd.concat([summary_df, pd.DataFrame([new_row])], ignore_index=True)
        
        # Store in dictionary
        summary_tables[region] = summary_df

    # Create 1x2 grid figure with custom width ratios
    fig, axs = plt.subplots(1, 2, figsize=(12, 4), gridspec_kw={'width_ratios': [1, 2]})

    # PLOT 1: Overall EP percentage (left panel)
    sns.barplot(
        x='Model', 
        y='EP_pct', 
        data=summary_tables['overall'], 
        ax=axs[0], 
        palette=palette,
        hue='Model',
        legend=False
    )
    axs[0].set_title(f'Overall Error Percentage (Patient {ptid})', fontsize=14, fontweight='bold')
    axs[0].set_ylabel('Error Percentage (%)', fontsize=12)
    axs[0].set_xlabel('')  # Remove x-axis label
    axs[0].tick_params(axis='both', labelsize=9)
    axs[0].set_ylim(0, 50)  # Adjusted y-axis range for percentage values

    # Add data labels
    for p in axs[0].patches:
        axs[0].annotate(f"{p.get_height():.1f}%", 
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='bottom', fontsize=11, fontweight='bold')

    # PLOT 2: EP percentage by Glycemic Region (right panel)
    # Create a new dataframe that combines the regional data for plotting
    regional_data = pd.concat([
        summary_tables['hypo'][['Model', 'EP_pct']].assign(Region='Hypo'),
        summary_tables['eu'][['Model', 'EP_pct']].assign(Region='Eu'),
        summary_tables['hyper'][['Model', 'EP_pct']].assign(Region='Hyper')
    ])

    sns.barplot(
        x='Region', 
        y='EP_pct', 
        hue='Model',
        data=regional_data, 
        ax=axs[1], 
        palette=palette
    )
    axs[1].set_title(f'Error Percentage Across Glycemic Regions (Patient {ptid})', fontsize=14, fontweight='bold')
    axs[1].set_ylabel('Error Percentage (%)', fontsize=12)
    axs[1].set_xlabel('Glycemic Region', fontsize=12)
    axs[1].tick_params(axis='both', labelsize=10)
    axs[1].set_ylim(0,50)  # Adjusted y-axis range for percentage values

    # Add data labels to the second chart
    for container in axs[1].containers:
        axs[1].bar_label(container, fmt='%.1f%%', fontsize=11, fontweight='bold')

    # Add legend to the right plot
    axs[1].legend(title='Model', fontsize=9, title_fontsize=10)

    # Remove top and right borders
    sns.despine(ax=axs[0])
    sns.despine(ax=axs[1])

    plt.tight_layout()
    plt.subplots_adjust(top=0.9)  # Adjust to make room for the title
    plt.show()


In [ ]:
for ptid in ptid_list:
    print(f"Patient ID: {ptid}")

    # Define custom colors for the models
    model1_color = (173 / 255, 29 / 255, 30 / 255)  # Without Feature Enhancement
    model2_color = (110 / 255, 180 / 255, 186 / 255)  # With Feature Enhancement
    palette = [model1_color, model2_color]

    # Create summary tables for overall, hypo, eu, and hyper glycaemic regions
    regions = ['overall', 'hypo', 'eu', 'hyper']
    summary_tables = {}

    # Get the dataframes for the current patient ID
    base_df = updated_base_df[ptid]
    fine_tuned_df = updated_fine_tuned_df[ptid]
    
    for region in regions:
        # Initialize empty DataFrame with specific dtypes to avoid warning
        summary_df = pd.DataFrame({
            'Model': pd.Series(dtype='object'),
            'AP': pd.Series(dtype='int64'), 
            'BE': pd.Series(dtype='int64'), 
            'EP': pd.Series(dtype='int64'), 
            'Count': pd.Series(dtype='int64'),
            'AP_pct': pd.Series(dtype='float64'), 
            'BE_pct': pd.Series(dtype='float64'), 
            'EP_pct': pd.Series(dtype='float64')
        })
        
        # Process each model
        for model, df in zip(['base model', 'fine-tuned models'], 
                            [base_df, fine_tuned_df]):
            
            # Filter for region if not overall
            if region != 'overall':
                region_df = df[df['glycemic_region'] == region]
            else:
                region_df = df
                
            # Calculate counts
            ap_count = region_df['AP'].sum()
            be_count = region_df['BE'].sum()
            ep_count = region_df['EP'].sum()
            total_count = len(region_df)
            
            # Create a new row as a dictionary and append it to the DataFrame
            new_row = {
                'Model': model, 
                'AP': ap_count, 
                'BE': be_count, 
                'EP': ep_count, 
                'Count': total_count,
                'AP:EP': ap_count / ep_count if ep_count != 0 else np.nan,
                'AP_pct': ap_count / total_count * 100, 
                'BE_pct': be_count / total_count * 100, 
                'EP_pct': ep_count / total_count * 100
            }
            summary_df = pd.concat([summary_df, pd.DataFrame([new_row])], ignore_index=True)
        
        # Store in dictionary
        summary_tables[region] = summary_df

    # Create 1x2 grid figure with custom width ratios
    fig, axs = plt.subplots(1, 2, figsize=(12, 4), gridspec_kw={'width_ratios': [1, 2]})

    # PLOT 1: Overall AP percentage (left panel) - changed from EP to AP
    sns.barplot(
        x='Model', 
        y='AP_pct', 
        data=summary_tables['overall'], 
        ax=axs[0], 
        palette=palette,
        hue='Model',
        legend=False
    )
    axs[0].set_title(f'Overall Accuracy Percentage (Patient {ptid})', fontsize=14, fontweight='bold')
    axs[0].set_ylabel('Accuracy Percentage (%)', fontsize=12)
    axs[0].set_xlabel('')  # Remove x-axis label
    axs[0].tick_params(axis='both', labelsize=9)
    axs[0].set_ylim(0, 100)  # Adjusted y-axis range for percentage values

    # Add data labels
    for p in axs[0].patches:
        axs[0].annotate(f"{p.get_height():.1f}%", 
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='bottom', fontsize=11, fontweight='bold')

    # PLOT 2: AP percentage by Glycemic Region (right panel) - changed from EP to AP
    # Create a new dataframe that combines the regional data for plotting
    regional_data = pd.concat([
        summary_tables['hypo'][['Model', 'AP_pct']].assign(Region='Hypo'),
        summary_tables['eu'][['Model', 'AP_pct']].assign(Region='Eu'),
        summary_tables['hyper'][['Model', 'AP_pct']].assign(Region='Hyper')
    ])

    sns.barplot(
        x='Region', 
        y='AP_pct', 
        hue='Model',
        data=regional_data, 
        ax=axs[1], 
        palette=palette
    )
    axs[1].set_title(f'Accuracy Percentage Across Glycemic Regions (Patient {ptid})', fontsize=14, fontweight='bold')
    axs[1].set_ylabel('Accuracy Percentage (%)', fontsize=12)
    axs[1].set_xlabel('Glycemic Region', fontsize=12)
    axs[1].tick_params(axis='both', labelsize=10)
    axs[1].set_ylim(0,100)  # Adjusted y-axis range for percentage values

    # Add data labels to the second chart
    for container in axs[1].containers:
        axs[1].bar_label(container, fmt='%.1f%%', fontsize=11, fontweight='bold')

    # Add legend to the right plot
    axs[1].legend(title='Model', fontsize=9, title_fontsize=10)

    # Remove top and right borders
    sns.despine(ax=axs[0])
    sns.despine(ax=axs[1])

    plt.tight_layout()
    plt.subplots_adjust(top=0.9)  # Adjust to make room for the title
    plt.show()


In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Compute AP% differences for each glycemic region
ptid_hypo_ap_diff = {}
ptid_eu_ap_diff = {}
ptid_hyper_ap_diff = {}

for ptid in ptid_list:
    # HYPO region
    base_hypo_df = updated_base_df[ptid][updated_base_df[ptid]['glycemic_region'] == 'hypo']
    fine_tuned_hypo_df = updated_fine_tuned_df[ptid][updated_fine_tuned_df[ptid]['glycemic_region'] == 'hypo']
    
    if len(base_hypo_df) > 0 and len(fine_tuned_hypo_df) > 0:
        base_hypo_ap_pct = base_hypo_df['AP'].sum() / len(base_hypo_df) * 100
        fine_tuned_hypo_ap_pct = fine_tuned_hypo_df['AP'].sum() / len(fine_tuned_hypo_df) * 100
        ptid_hypo_ap_diff[ptid] = fine_tuned_hypo_ap_pct - base_hypo_ap_pct
    
    # EU region
    base_eu_df = updated_base_df[ptid][updated_base_df[ptid]['glycemic_region'] == 'eu']
    fine_tuned_eu_df = updated_fine_tuned_df[ptid][updated_fine_tuned_df[ptid]['glycemic_region'] == 'eu']
    
    if len(base_eu_df) > 0 and len(fine_tuned_eu_df) > 0:
        base_eu_ap_pct = base_eu_df['AP'].sum() / len(base_eu_df) * 100
        fine_tuned_eu_ap_pct = fine_tuned_eu_df['AP'].sum() / len(fine_tuned_eu_df) * 100
        ptid_eu_ap_diff[ptid] = fine_tuned_eu_ap_pct - base_eu_ap_pct
    
    # HYPER region
    base_hyper_df = updated_base_df[ptid][updated_base_df[ptid]['glycemic_region'] == 'hyper']
    fine_tuned_hyper_df = updated_fine_tuned_df[ptid][updated_fine_tuned_df[ptid]['glycemic_region'] == 'hyper']
    
    if len(base_hyper_df) > 0 and len(fine_tuned_hyper_df) > 0:
        base_hyper_ap_pct = base_hyper_df['AP'].sum() / len(base_hyper_df) * 100
        fine_tuned_hyper_ap_pct = fine_tuned_hyper_df['AP'].sum() / len(fine_tuned_hyper_df) * 100
        ptid_hyper_ap_diff[ptid] = fine_tuned_hyper_ap_pct - base_hyper_ap_pct



# Create 3x1 plot
fig, axs = plt.subplots(1, 3, figsize=(14, 10))

model1_color = (173 / 255, 29 / 255, 30 / 255)
model2_color = (110 / 255, 180 / 255, 186 / 255)

# ---- HYPOTHETIC REGION ----
# Prepare significance dict
hypo_significance_dict = {
    row['Patient ID']: row['Significant']
    for _, row in hypo_chi2_summary.iterrows()
}
hypo_df = pd.DataFrame({
    'Patient ID': [f"Patient {ptid}" for ptid in sorted(ptid_hypo_ap_diff.keys())],
    'AP Percentage Difference': [ptid_hypo_ap_diff[ptid] for ptid in sorted(ptid_hypo_ap_diff.keys())],
    'Is_Significant': [hypo_significance_dict.get(ptid, False) for ptid in sorted(ptid_hypo_ap_diff.keys())]
}).sort_values('AP Percentage Difference')

hypo_colors = [model2_color if val > 0 else model1_color for val in hypo_df['AP Percentage Difference']]

bars_hypo = axs[0].barh(hypo_df['Patient ID'], hypo_df['AP Percentage Difference'], color=hypo_colors)
axs[0].set_title('Hypoglycemia', fontsize=16, fontweight='bold')
axs[0].axvline(0, color='black', linestyle='-', linewidth=1)
axs[0].set_xlim(-25, 25)
axs[0].tick_params(axis='both', labelsize=14)

# Annotate bars with value and asterisk
for i, bar in enumerate(bars_hypo):
    width = bar.get_width()
    label_x_pos = width + 0.5 if width > 0 else width - 0.5
    asterisk = '*' if hypo_df['Is_Significant'].iloc[i] else ''
    axs[0].text(label_x_pos, bar.get_y() + bar.get_height()/2,
                f'{abs(width):.1f}%{asterisk}', va='center',
                ha='left' if width > 0 else 'right', fontsize=12)

# ---- EUGLYCEMIC REGION ----
eu_significance_dict = {
    row['Patient ID']: row['Significant']
    for _, row in eu_chi2_summary.iterrows()
}

eu_df = pd.DataFrame({
    'Patient ID': [f"Patient {ptid}" for ptid in sorted(ptid_eu_ap_diff.keys())],
    'AP Percentage Difference': [-ptid_eu_ap_diff[ptid] for ptid in sorted(ptid_eu_ap_diff.keys())],
    'Is_Significant': [eu_significance_dict.get(ptid, False) for ptid in sorted(ptid_eu_ap_diff.keys())]
}).set_index('Patient ID').reindex(hypo_df['Patient ID']).reset_index()

eu_colors = [model2_color if val > 0 else model1_color for val in eu_df['AP Percentage Difference']]

bars_eu = axs[1].barh(eu_df['Patient ID'], eu_df['AP Percentage Difference'], color=eu_colors)
axs[1].set_title('Euglycemia', fontsize=16, fontweight='bold')
axs[1].axvline(0, color='black', linestyle='-', linewidth=1)
axs[1].set_xlim(-25, 25)
axs[1].set_yticks([])
axs[1].tick_params(axis='x', labelsize=14)

for i, bar in enumerate(bars_eu):
    width = bar.get_width()
    label_x_pos = width + 0.5 if width > 0 else width - 0.5
    asterisk = '*' if eu_df['Is_Significant'].iloc[i] else ''
    axs[1].text(label_x_pos, bar.get_y() + bar.get_height()/2,
                f'{abs(width):.1f}%{asterisk}', va='center',
                ha='left' if width > 0 else 'right', fontsize=12)

# ---- HYPERGLYCEMIC REGION ----
hyper_significance_dict = {
    row['Patient ID']: row['Significant']
    for _, row in hyper_chi2_summary.iterrows()
}

hyper_df = pd.DataFrame({
    'Patient ID': [f"Patient {ptid}" for ptid in sorted(ptid_hyper_ap_diff.keys())],
    'AP Percentage Difference': [-ptid_hyper_ap_diff[ptid] for ptid in sorted(ptid_hyper_ap_diff.keys())],
    'Is_Significant': [hyper_significance_dict.get(ptid, False) for ptid in sorted(ptid_hyper_ap_diff.keys())]
}).set_index('Patient ID').reindex(hypo_df['Patient ID']).reset_index()

hyper_colors = [model2_color if val > 0 else model1_color for val in hyper_df['AP Percentage Difference']]

bars_hyper = axs[2].barh(hyper_df['Patient ID'], hyper_df['AP Percentage Difference'], color=hyper_colors)
axs[2].set_title('Hyperglycemia', fontsize=16, fontweight='bold')
axs[2].axvline(0, color='black', linestyle='-', linewidth=1)
axs[2].set_xlim(-25, 25)
axs[2].set_yticks([])

for i, bar in enumerate(bars_hyper):
    width = bar.get_width()
    label_x_pos = width + 0.5 if width > 0 else width - 0.5
    asterisk = '*' if hyper_df['Is_Significant'].iloc[i] else ''
    axs[2].text(label_x_pos, bar.get_y() + bar.get_height()/2,
                f'{abs(width):.1f}%{asterisk}', va='center',
                ha='left' if width > 0 else 'right', fontsize=12)

# Styling
for ax in axs:
    ax.set_xlabel('AP Percentage Difference (%)', fontsize=12)
    sns.despine(ax=ax)

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.text(xlim[0]+0.5, ylim[0] - 1.25, 'Population\n JPFormer better', 
            color=model1_color, fontsize=12, fontweight='bold', ha='left', va='top')
    ax.text(xlim[1]-0.5, ylim[0] - 1.25, 'Personalised\n JPFormer better', 
            color=model2_color, fontsize=12, fontweight='bold', ha='right', va='top')

axs[1].spines['left'].set_visible(False)
axs[2].spines['left'].set_visible(False)

plt.suptitle('AP% Absolute Difference by Glycemic Region:\nBase Model vs Fine-tuned Model', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.subplots_adjust(hspace=0.3)
plt.show()

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Compute EP% differences for each glycemic region
ptid_hypo_ep_diff = {}
ptid_eu_ep_diff = {}
ptid_hyper_ep_diff = {}

for ptid in ptid_list:
    # HYPO region
    base_hypo_df = updated_base_df[ptid][updated_base_df[ptid]['glycemic_region'] == 'hypo']
    fine_tuned_hypo_df = updated_fine_tuned_df[ptid][updated_fine_tuned_df[ptid]['glycemic_region'] == 'hypo']
    
    if len(base_hypo_df) > 0 and len(fine_tuned_hypo_df) > 0:
        base_hypo_ep_pct = base_hypo_df['EP'].sum() / len(base_hypo_df) * 100
        fine_tuned_hypo_ep_pct = fine_tuned_hypo_df['EP'].sum() / len(fine_tuned_hypo_df) * 100
        ptid_hypo_ep_diff[ptid] = (fine_tuned_hypo_ep_pct - base_hypo_ep_pct)*-1
    
    # EU region
    base_eu_df = updated_base_df[ptid][updated_base_df[ptid]['glycemic_region'] == 'eu']
    fine_tuned_eu_df = updated_fine_tuned_df[ptid][updated_fine_tuned_df[ptid]['glycemic_region'] == 'eu']
    
    if len(base_eu_df) > 0 and len(fine_tuned_eu_df) > 0:
        base_eu_ep_pct = base_eu_df['EP'].sum() / len(base_eu_df) * 100
        fine_tuned_eu_ep_pct = fine_tuned_eu_df['EP'].sum() / len(fine_tuned_eu_df) * 100
        ptid_eu_ep_diff[ptid] = fine_tuned_eu_ep_pct - base_eu_ep_pct
    
    # HYPER region
    base_hyper_df = updated_base_df[ptid][updated_base_df[ptid]['glycemic_region'] == 'hyper']
    fine_tuned_hyper_df = updated_fine_tuned_df[ptid][updated_fine_tuned_df[ptid]['glycemic_region'] == 'hyper']
    
    if len(base_hyper_df) > 0 and len(fine_tuned_hyper_df) > 0:
        base_hyper_ep_pct = base_hyper_df['EP'].sum() / len(base_hyper_df) * 100
        fine_tuned_hyper_ep_pct = fine_tuned_hyper_df['EP'].sum() / len(fine_tuned_hyper_df) * 100
        ptid_hyper_ep_diff[ptid] = fine_tuned_hyper_ep_pct - base_hyper_ep_pct

# Create 3x1 plot
fig, axs = plt.subplots(1, 3, figsize=(14, 10))

model1_color = (173 / 255, 29 / 255, 30 / 255)
model2_color = (110 / 255, 180 / 255, 186 / 255)

# ---- HYPOTHETIC REGION ----
hypo_significance_dict = {
    row['Patient ID']: row['Significant']
    for _, row in hypo_chi2_summary.iterrows()
}
hypo_df = pd.DataFrame({
    'Patient ID': [f"Patient {ptid}" for ptid in sorted(ptid_hypo_ep_diff.keys())],
    'EP Percentage Difference': [ptid_hypo_ep_diff[ptid] for ptid in sorted(ptid_hypo_ep_diff.keys())],
    'Is_Significant': [hypo_significance_dict.get(ptid, False) for ptid in sorted(ptid_hypo_ep_diff.keys())]
}).sort_values('EP Percentage Difference')

hypo_colors = [model2_color if val > 0 else model1_color for val in hypo_df['EP Percentage Difference']]

bars_hypo = axs[0].barh(hypo_df['Patient ID'], hypo_df['EP Percentage Difference'], color=hypo_colors)
axs[0].set_title('Hypoglycemia', fontsize=16, fontweight='bold')
axs[0].axvline(0, color='black', linestyle='-', linewidth=1)
axs[0].set_xlim(-25, 25)
axs[0].tick_params(axis='both', labelsize=14)

for i, bar in enumerate(bars_hypo):
    width = bar.get_width()
    label_x_pos = width + 0.5 if width > 0 else width - 0.5
    asterisk = '*' if hypo_df['Is_Significant'].iloc[i] else ''
    axs[0].text(label_x_pos, bar.get_y() + bar.get_height()/2,
                f'{abs(width):.1f}%{asterisk}', va='center',
                ha='left' if width > 0 else 'right', fontsize=12)

# ---- EUGLYCEMIC REGION ----
eu_significance_dict = {
    row['Patient ID']: row['Significant']
    for _, row in eu_chi2_summary.iterrows()
}

eu_df = pd.DataFrame({
    'Patient ID': [f"Patient {ptid}" for ptid in sorted(ptid_eu_ep_diff.keys())],
    'EP Percentage Difference': [-ptid_eu_ep_diff[ptid] for ptid in sorted(ptid_eu_ep_diff.keys())],
    'Is_Significant': [eu_significance_dict.get(ptid, False) for ptid in sorted(ptid_eu_ep_diff.keys())]
}).set_index('Patient ID').reindex(hypo_df['Patient ID']).reset_index()

eu_colors = [model2_color if val > 0 else model1_color for val in eu_df['EP Percentage Difference']]

bars_eu = axs[1].barh(eu_df['Patient ID'], eu_df['EP Percentage Difference'], color=eu_colors)
axs[1].set_title('Euglycemia', fontsize=16, fontweight='bold')
axs[1].axvline(0, color='black', linestyle='-', linewidth=1)
axs[1].set_xlim(-25, 25)
axs[1].set_yticks([])
axs[1].tick_params(axis='x', labelsize=14)

for i, bar in enumerate(bars_eu):
    width = bar.get_width()
    label_x_pos = width + 0.5 if width > 0 else width - 0.5
    asterisk = '*' if eu_df['Is_Significant'].iloc[i] else ''
    axs[1].text(label_x_pos, bar.get_y() + bar.get_height()/2,
                f'{abs(width):.1f}%{asterisk}', va='center',
                ha='left' if width > 0 else 'right', fontsize=12)

# ---- HYPERGLYCEMIC REGION ----
hyper_significance_dict = {
    row['Patient ID']: row['Significant']
    for _, row in hyper_chi2_summary.iterrows()
}

hyper_df = pd.DataFrame({
    'Patient ID': [f"Patient {ptid}" for ptid in sorted(ptid_hyper_ep_diff.keys())],
    'EP Percentage Difference': [-ptid_hyper_ep_diff[ptid] for ptid in sorted(ptid_hyper_ep_diff.keys())],
    'Is_Significant': [hyper_significance_dict.get(ptid, False) for ptid in sorted(ptid_hyper_ep_diff.keys())]
}).set_index('Patient ID').reindex(hypo_df['Patient ID']).reset_index()

hyper_colors = [model2_color if val > 0 else model1_color for val in hyper_df['EP Percentage Difference']]

bars_hyper = axs[2].barh(hyper_df['Patient ID'], hyper_df['EP Percentage Difference'], color=hyper_colors)
axs[2].set_title('Hyperglycemia', fontsize=16, fontweight='bold')
axs[2].axvline(0, color='black', linestyle='-', linewidth=1)
axs[2].set_xlim(-25, 25)
axs[2].set_yticks([])

for i, bar in enumerate(bars_hyper):
    width = bar.get_width()
    label_x_pos = width + 0.5 if width > 0 else width - 0.5
    asterisk = '*' if hyper_df['Is_Significant'].iloc[i] else ''
    axs[2].text(label_x_pos, bar.get_y() + bar.get_height()/2,
                f'{abs(width):.1f}%{asterisk}', va='center',
                ha='left' if width > 0 else 'right', fontsize=12)

# Styling
for ax in axs:
    ax.set_xlabel('EP Percentage Difference (%)', fontsize=12)
    sns.despine(ax=ax)

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.text(xlim[0]+0.5, ylim[0] - 1.25, 'Population\n JPFormer better', 
            color=model1_color, fontsize=12, fontweight='bold', ha='left', va='top')
    ax.text(xlim[1]-0.5, ylim[0] - 1.25, 'Personalised\n JPFormer better', 
            color=model2_color, fontsize=12, fontweight='bold', ha='right', va='top')

axs[1].spines['left'].set_visible(False)
axs[2].spines['left'].set_visible(False)

plt.suptitle('EP% Absolute Difference by Glycemic Region:\nBase Model vs Fine-tuned Model', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.subplots_adjust(hspace=0.3)
plt.show()


In [ ]:


# First, calculate the AP% and EP% differences
ptid_overall_ap_diff = {}
ptid_overall_ep_diff = {}

for ptid in ptid_list:
    # Get overall data
    base_df = updated_base_df[ptid]
    fine_tuned_df = updated_fine_tuned_df[ptid]
    
    base_total = len(base_df)
    fine_tuned_total = len(fine_tuned_df)
    
    if base_total > 0 and fine_tuned_total > 0:
        base_ap_pct = base_df['AP'].sum() / base_total * 100
        fine_tuned_ap_pct = fine_tuned_df['AP'].sum() / fine_tuned_total * 100
        ap_pct_diff = fine_tuned_ap_pct - base_ap_pct
        ptid_overall_ap_diff[ptid] = ap_pct_diff
        
        base_ep_pct = base_df['EP'].sum() / base_total * 100
        fine_tuned_ep_pct = fine_tuned_df['EP'].sum() / fine_tuned_total * 100
        ep_pct_diff = fine_tuned_ep_pct - base_ep_pct
        ptid_overall_ep_diff[ptid] = ep_pct_diff

# Create more detailed DataFrames for tables
comparison_df = pd.DataFrame(index=[f"Patient {ptid}" for ptid in ptid_list])

# Add AP metrics
comparison_df['AP% Base Model'] = [round(updated_base_df[ptid]['AP'].sum() / len(updated_base_df[ptid]) * 100, 2) if len(updated_base_df[ptid]) > 0 else 0 for ptid in ptid_list]
comparison_df['AP% Fine-tuned'] = [round(updated_fine_tuned_df[ptid]['AP'].sum() / len(updated_fine_tuned_df[ptid]) * 100, 2) if len(updated_fine_tuned_df[ptid]) > 0 else 0 for ptid in ptid_list]
comparison_df['AP% Difference'] = comparison_df['AP% Fine-tuned'] - comparison_df['AP% Base Model']

# Add EP metrics
comparison_df['EP% Base Model'] = [round(updated_base_df[ptid]['EP'].sum() / len(updated_base_df[ptid]) * 100, 2) if len(updated_base_df[ptid]) > 0 else 0 for ptid in ptid_list]
comparison_df['EP% Fine-tuned'] = [round(updated_fine_tuned_df[ptid]['EP'].sum() / len(updated_fine_tuned_df[ptid]) * 100, 2) if len(updated_fine_tuned_df[ptid]) > 0 else 0 for ptid in ptid_list]
comparison_df['EP% Difference'] = comparison_df['EP% Fine-tuned'] - comparison_df['EP% Base Model']

# Sort by AP% Difference
overall_comparison_df = comparison_df.sort_values('AP% Difference', ascending=False)

# Style the dataframe for better visualization
overall_styled_df = overall_comparison_df.style.background_gradient(subset=['AP% Difference', 'EP% Difference'], 
                                                  cmap='RdYlGn', 
                                                  vmin=-0.6, 
                                                  vmax=0.6)

# Format to show percentage signs
styled_df = styled_df.format("{:.2f}%")

# Add a title
print("AP% and EP% Metrics: PopJPFormer vs PersJPFormer")

# Display the table
display(overall_styled_df)


In [ ]:


# Create a summary table showing the percentage change in hypoglycemia detection
hypoglycemia_summary = pd.DataFrame({
    'Patient ID': [f"Patient {ptid}" for ptid in sorted_ptids],
    'Base AP%': [round(updated_base_df[ptid][updated_base_df[ptid]['glycemic_region'] == 'hypo']['AP'].mean() * 100, 1) 
                for ptid in sorted_ptids],
    'Fine-tuned AP%': [round(updated_fine_tuned_df[ptid][updated_fine_tuned_df[ptid]['glycemic_region'] == 'hypo']['AP'].mean() * 100, 1) 
                      for ptid in sorted_ptids],
    'AP% Change': [round(ptid_hypo_ap_diff[ptid], 1) for ptid in sorted_ptids],
    'Base EP%': [round(updated_base_df[ptid][updated_base_df[ptid]['glycemic_region'] == 'hypo']['EP'].mean() * 100, 1) 
                for ptid in sorted_ptids],
    'Fine-tuned EP%': [round(updated_fine_tuned_df[ptid][updated_fine_tuned_df[ptid]['glycemic_region'] == 'hypo']['EP'].mean() * 100, 1) 
                      for ptid in sorted_ptids],
    'EP% Change': [round(-ptid_hypo_ep_diff[ptid], 1) for ptid in sorted_ptids]
})

# Style the DataFrame for better visualization
hypo_styled_summary = hypoglycemia_summary

# Display the table
print("\nHypoglycemia AP and EP% Performance by Patient")
display(hypo_styled_summary)


## Correlation Analysis

### Hypoglycaemia

In [ ]:
hypo_styled_summary


In [ ]:
hypo_styled_summary['EP% Improvement'] = hypo_styled_summary['EP% Change']*-1

In [ ]:
from scipy.stats import pearsonr, spearmanr
import seaborn as sns

import matplotlib.pyplot as plt

def compute_correlation(x, y):
    pearson_corr, pearson_p = pearsonr(x, y)
    spearman_corr, spearman_p = spearmanr(x, y)
    return {
        "Pearson Coefficient": pearson_corr,
        "Pearson p-value": pearson_p,
        "Spearman Coefficient": spearman_corr,
        "Spearman p-value": spearman_p
    }

# Use the original DataFrame instead of the styled version
base_ap_change_corr = compute_correlation(
    hypoglycemia_summary['Base AP%'], 
    hypoglycemia_summary['AP% Change']
)
base_ep_change_corr = compute_correlation(
    hypoglycemia_summary['Base EP%'], 
    hypoglycemia_summary['EP% Improvement']
)

# plotting
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Scatter plot for Base AP% vs AP% Change
axs[0].scatter(hypo_styled_summary['Base AP%'], hypo_styled_summary['AP% Change'], 
               color=model1_color, alpha=0.7)
axs[0].set_title('AP% Improvement', fontsize=16, fontweight='bold', pad=20)
axs[0].set_xlabel('Base AP%', fontsize=14)
axs[0].set_ylabel('AP% Change', fontsize=14)
axs[0].tick_params(axis='both', labelsize=14)

# Add regression line without confidence interval (ci=None)
sns.regplot(x=hypo_styled_summary['Base AP%'], 
            y=hypo_styled_summary['AP% Change'], 
            ax=axs[0], 
            scatter=False, 
            color='black', 
            ci=None,
            line_kws={'linestyle': '--', 'linewidth': 1})

# Scatter plot for Base EP% vs EP% Change
axs[1].scatter(hypo_styled_summary['Base EP%'], hypo_styled_summary['EP% Improvement'], 
               color=model1_color, alpha=0.7)
axs[1].set_title('EP% Improvement', fontsize=16, fontweight='bold', pad=20)
axs[1].set_xlabel('Base EP%', fontsize=14)
axs[1].set_ylabel('EP% Improvement', fontsize=14)
axs[1].tick_params(axis='both', labelsize=14)

# Add regression line without confidence interval (ci=None)
sns.regplot(x=hypo_styled_summary['Base EP%'], 
            y=hypo_styled_summary['EP% Improvement'], 
            ax=axs[1], 
            scatter=False, 
            color='black', 
            ci=None,
            line_kws={'linestyle': '--', 'linewidth': 1})

# Add correlation coefficients to the plots
axs[0].text(0.95, 0.95, f"Pearson: {base_ap_change_corr['Spearman Coefficient']:.2f}\np-value: {base_ap_change_corr['Spearman p-value']:.3f}",
            transform=axs[0].transAxes, 
            fontsize=12, 
            verticalalignment='top', 
            horizontalalignment='right',
            bbox=dict(facecolor='white', alpha=0.5))
axs[1].text(0.05, 0.95, f"Pearson: {base_ep_change_corr['Spearman Coefficient']:.2f}\np-value: {base_ep_change_corr['Spearman p-value']:.3f}",
            transform=axs[1].transAxes, 
            fontsize=12, 
            verticalalignment='top', 
            bbox=dict(facecolor='white', alpha=0.5))

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Create the DataFrame
data = {
    "Patient ID": [584, 575, 563, 559, 540, 596, 588, 570, 591, 567, 552, 544],
    "Total Slices": [582, 3704, 2142, 1620, 5082, 876, 846, 1140, 1674, 2412, 1974, 1134]
}

df = pd.DataFrame(data)

In [ ]:
hypo_styled_summary['Total Traing Slices'] = df['Total Slices']
hypo_styled_summary['Total Traing Slices'] = hypo_styled_summary['Total Traing Slices'].astype(int)

In [ ]:
from scipy.stats import pearsonr, spearmanr
import seaborn as sns

import matplotlib.pyplot as plt

def compute_correlation(x, y):
    pearson_corr, pearson_p = pearsonr(x, y)
    spearman_corr, spearman_p = spearmanr(x, y)
    return {
        "Pearson Coefficient": pearson_corr,
        "Pearson p-value": pearson_p,
        "Spearman Coefficient": spearman_corr,
        "Spearman p-value": spearman_p
    }

# Use the original DataFrame instead of the styled version
base_ap_change_corr = compute_correlation(
    hypoglycemia_summary['Total Traing Slices'], 
    hypoglycemia_summary['AP% Change']
)
base_ep_change_corr = compute_correlation(
    hypoglycemia_summary['Total Traing Slices'], 
    hypoglycemia_summary['EP% Improvement']
)

# plotting
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Scatter plot for Base AP% vs AP% Change
axs[0].scatter(hypo_styled_summary['Total Traing Slices'], hypo_styled_summary['AP% Change'], 
               color=model1_color, alpha=0.7)
axs[0].set_title('AP% Improvement', fontsize=16, fontweight='bold', pad=20)
axs[0].set_xlabel('Total Traing Slices', fontsize=14)
axs[0].set_ylabel('AP% Change', fontsize=14)
axs[0].tick_params(axis='both', labelsize=14)

# Add regression line without confidence interval (ci=None)
sns.regplot(x=hypo_styled_summary['Total Traing Slices'], 
            y=hypo_styled_summary['AP% Change'], 
            ax=axs[0], 
            scatter=False, 
            color='black', 
            ci=None,
            line_kws={'linestyle': '--', 'linewidth': 1})

# Scatter plot for Base EP% vs EP% Change
axs[1].scatter(hypo_styled_summary['Total Traing Slices'], hypo_styled_summary['EP% Improvement'], 
               color=model1_color, alpha=0.7)
axs[1].set_title('EP% Improvement', fontsize=16, fontweight='bold', pad=20)
axs[1].set_xlabel('Total Traing Slices', fontsize=14)
axs[1].set_ylabel('EP% Improvement', fontsize=14)
axs[1].tick_params(axis='both', labelsize=14)

# Add regression line without confidence interval (ci=None)
sns.regplot(x=hypo_styled_summary['Total Traing Slices'], 
            y=hypo_styled_summary['EP% Improvement'], 
            ax=axs[1], 
            scatter=False, 
            color='black', 
            ci=None,
            line_kws={'linestyle': '--', 'linewidth': 1})

# Add correlation coefficients to the plots
axs[0].text(0.95, 0.95, f"Pearson: {base_ap_change_corr['Spearman Coefficient']:.2f}\np-value: {base_ap_change_corr['Spearman p-value']:.3f}",
            transform=axs[0].transAxes, 
            fontsize=12, 
            verticalalignment='top', 
            horizontalalignment='right',
            bbox=dict(facecolor='white', alpha=0.5))
axs[1].text(0.05, 0.95, f"Pearson: {base_ep_change_corr['Spearman Coefficient']:.2f}\np-value: {base_ep_change_corr['Spearman p-value']:.3f}",
            transform=axs[1].transAxes, 
            fontsize=12, 
            verticalalignment='top', 
            bbox=dict(facecolor='white', alpha=0.5))

# Adjust layout
plt.tight_layout()
plt.show()